In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
pd.options.display.max_colwidth = 500
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [2]:
NJOBS = 6
RANDOM = 99

Зарузим датафрейм и изучим его поподробнее.

In [3]:
df = pd.read_csv("crx.data", header=None)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null object
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null int64
15    690 non-null object
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [6]:
df[15].value_counts()

-    383
+    307
Name: 15, dtype: int64


Видим, что в датасете есть числовые (**int64**, **float64**) и нечисловые признаки (**object**). Первые 14 столбцов – признаки, а 15 содержит целевую переменную, которую мы и будет прогнозировать (одобрена или нет кредитная карта для конкретного клиента). В исходных данных 307 заявок было одобрено, 383 – отклонено.  
Если внимательнее изучить данные, то можно обнаружить, что в них присутствуют значения, которые представлены, как "**?**". Судя по всему, это пропуски в данных.

In [7]:
df.isin(['?']).sum(axis=0)

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

"**?**" присутствуют в столбцах, как с числовыми (1), так и нечисловыми признаками (0, 3-6, 13). От пропусков в данных необходимо избавиться, т.к. они могут сильно влиять на точность моделей. Для начала заменим все "**?**" на **NaN** при помощи метода **replace**.

In [8]:
df[1] = df[1].astype('float64') 
df.replace('?', np.nan, inplace=True)

In [9]:
df.isna().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

Создадим pipeline с предобработкой данных

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X, y = df.iloc[:,0:15] , df.iloc[:,15]
y = y.astype('category').cat.codes

cat_columns = X.dtypes[X.dtypes == 'object'].index
num_columns = X.dtypes[X.dtypes != 'object'].index

num_pipe = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('mms', MinMaxScaler(feature_range=(0, 1)))
])
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])
transformer = ColumnTransformer(transformers=
                                [('num', num_pipe, num_columns),
                                 ('cat', cat_pipe, cat_columns)])

Разделим нашу выборку на тестовую (на которой будем обучать модели) и тренировочную (на которой будем проверять результаты) при помощи функции **train_test_split** в соотношении 70/30 и зафиксируем **seed** для воспроизводимости результатов.

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=RANDOM)

Обучаем модели

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
res = {}

1) Логистическая регрессия

In [14]:
logreg_pipe = make_pipeline(transformer, LogisticRegression(random_state=RANDOM, n_jobs=NJOBS))
logreg_params_grid = {'logisticregression__C': np.logspace(-4, 2, 20), 
                      'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear'],
                      'logisticregression__class_weight': ['balanced', None]
                     }
logreg_grid = RandomizedSearchCV(logreg_pipe, logreg_params_grid, scoring='f1', random_state=RANDOM) 
logreg_grid.fit(X_train, y_train)
print('Лучшие параметры:', logreg_grid.best_params_)
print('F-мера на перекрестной проверке:', logreg_grid.best_score_)
print('F-мера логистической регрессии на тестовом наборе:', logreg_grid.score(X_test, y_test))
res['LogReg'] = logreg_grid.score(X_test, y_test)

Лучшие параметры: {'logisticregression__solver': 'newton-cg', 'logisticregression__class_weight': None, 'logisticregression__C': 48.32930238571752}
F-мера на перекрестной проверке: 0.8809905513773048
F-мера логистической регрессии на тестовом наборе: 0.8765957446808511


2) Дерево решений

In [15]:
dtc_pipe = make_pipeline(transformer, DecisionTreeClassifier(random_state=RANDOM))
dtc_params_grid = {'decisiontreeclassifier__min_samples_split': range(2, 200, 5), 
                  'decisiontreeclassifier__criterion': ['gini', 'entropy'],
                   'decisiontreeclassifier__max_depth': range(1, 35),
                  'decisiontreeclassifier__class_weight': ['balanced', None],
                  'decisiontreeclassifier__max_features': ['auto', None, 'log2']}
dtc_grid = RandomizedSearchCV(dtc_pipe, dtc_params_grid, scoring='f1', random_state=RANDOM)  
dtc_grid.fit(X_train, y_train)
print('Лучшие параметры:', dtc_grid.best_params_)
print('F-мера на перекрестной проверке:', dtc_grid.best_score_)
print('F-мера дерева решений на тестовом наборе:', dtc_grid.score(X_test, y_test))
res['DTC'] = dtc_grid.score(X_test, y_test)

Лучшие параметры: {'decisiontreeclassifier__min_samples_split': 87, 'decisiontreeclassifier__max_features': None, 'decisiontreeclassifier__max_depth': 11, 'decisiontreeclassifier__criterion': 'entropy', 'decisiontreeclassifier__class_weight': None}
F-мера на перекрестной проверке: 0.853534582310326
F-мера дерева решений на тестовом наборе: 0.8514056224899599


3) Случайный лес

In [16]:
rfc_pipe = make_pipeline(transformer, RandomForestClassifier(random_state=RANDOM, n_jobs=NJOBS))
rfc_params_grid = {'randomforestclassifier__min_samples_split': range(2, 100, 5), 
                   'randomforestclassifier__n_estimators': range(50, 200, 5), 
                   'randomforestclassifier__criterion': ['gini', 'entropy'],
                    'randomforestclassifier__max_depth': range(1, 35),
                   'randomforestclassifier__class_weight': ['balanced', None],
                   'randomforestclassifier__max_features': ['auto', None, 'log2']}
rfc_grid = RandomizedSearchCV(rfc_pipe, rfc_params_grid, scoring='f1', random_state=RANDOM)  
rfc_grid.fit(X_train, y_train)
print('Лучшие параметры:', rfc_grid.best_params_)
print('F-мера на перекрестной проверке:', rfc_grid.best_score_)
print('F-мера случайного леса на тестовом наборе:', rfc_grid.score(X_test, y_test))
res['RFC'] = rfc_grid.score(X_test, y_test)

Лучшие параметры: {'randomforestclassifier__n_estimators': 75, 'randomforestclassifier__min_samples_split': 32, 'randomforestclassifier__max_features': 'auto', 'randomforestclassifier__max_depth': 22, 'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__class_weight': None}
F-мера на перекрестной проверке: 0.8905675084259512
F-мера случайного леса на тестовом наборе: 0.8934426229508197


4) Метод опорных векторов

In [17]:
svc_pipe = make_pipeline(transformer, svm.SVC(random_state=RANDOM))
svc_params_grid = {'svc__C': np.logspace(-4, 2, 20), 
                   'svc__gamma': ['scale', 'auto'],
                   'svc__class_weight': ['balanced', None],
                   'svc__kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
svc_grid = RandomizedSearchCV(svc_pipe, svc_params_grid, scoring='f1', random_state=RANDOM)  
svc_grid.fit(X_train, y_train)
print('Лучшие параметры:', svc_grid.best_params_)
print('F-мера на перекрестной проверке:', svc_grid.best_score_)
print('F-мера метода опорных векторов на тестовом наборе:', svc_grid.score(X_test, y_test))
res['SVC'] = svc_grid.score(X_test, y_test)

Лучшие параметры: {'svc__kernel': 'rbf', 'svc__gamma': 'scale', 'svc__class_weight': None, 'svc__C': 0.06951927961775606}
F-мера на перекрестной проверке: 0.887491855673543
F-мера метода опорных векторов на тестовом наборе: 0.8813559322033899


In [18]:
res

{'LogReg': 0.8765957446808511,
 'DTC': 0.8514056224899599,
 'RFC': 0.8934426229508197,
 'SVC': 0.8813559322033899}

Посмотрим на матрицу ошибок лучшей модели

In [23]:
from sklearn.metrics import confusion_matrix
y_pred = logreg_grid.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 75,  13],
       [ 16, 103]], dtype=int64)